In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import tensorflow as tf



Using TensorFlow backend.
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\rahul\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
model=InceptionV3(weights="imagenet")
model_new=Model(model.input,model.layers[-2].output)

max_length=72

ixtoword=load(open('ixtoword.pkl','rb'))
wordtoix=load(open('wordtoix.pkl','rb'))

nlp_model=tf.keras.models.load_model('./model_weights/final_model.h5')
nlp_model.compile(loss='categorical_crossentropy',optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.0001))




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
def preprocess(image_path):
    img=image.load_img(image_path,target_size=(299,299))
    x=image.img_to_array(img)
    x=np.expand_dims(x,axis=0)
    x=preprocess_input(x)
    return x

In [4]:
def encode(image):
    image=preprocess(image)
    fea_vec=model_new.predict(image)
    fea_vec=np.reshape(fea_vec,fea_vec.shape[1])
    return fea_vec

In [5]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = nlp_model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [6]:
test_folder='./test/'
test_images=glob.glob(test_folder+'*.jpg')


In [10]:
start=time()
encoding_test={}
for img in test_images:
    encoding_test[img[len(test_folder):]]=encode(img)
    #print("time tken in second=",time()-start)


In [ ]:
pics=list(encoding_test.keys())
for pic in pics:
    image=encoding_test[pic].reshape((1,2048))
    print(greedySearch(image))
    print('\n')

In [20]:
new_encoding_test={}
pic_name='1440465.jpg'
new_encoding_test[pic_name]=encode(pic_name)
image=new_encoding_test[pic_name].reshape((1,2048))
greedySearch(image)